In [ ]:
import cv2
import numpy as np

def find_orb_features(image):
  # Convert image to grayscale
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Initiate ORB detector
  orb = cv2.ORB_create()

  # Find keypoints and descriptors
  kp, descriptors = orb.detectAndCompute(gray, None)

  return kp, descriptors

def match_features(descriptors1, descriptors2, ratio=0.8):
  # Create Brute Force Matcher object
  matcher = cv2.BFMatcher(cv2.NORM_HAMMING)

  # Perform matching
  matches = matcher.knnMatch(descriptors1, descriptors2, k=2)

  # Filter good matches based on Lowe's ratio test
  good_matches = []
  for m, n in matches:
    if m.distance < ratio * n.distance:
      good_matches.append(m)

  return good_matches

def get_homography(kp1, kp2, matches):
  # Convert keypoints to numpy arrays
  src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
  dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

  # Find homography using RANSAC
  M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

  return M

def warp_perspective(img, M):
  # Get image height and width
  h, w = img.shape[:2]

  # Find corners of the image in the destination image
  pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
  dst = cv2.perspectiveTransform(pts, M)

  # Find minimum and maximum points
  x_min = np.min(dst[:, 0, 0])
  y_min = np.min(dst[:, 0, 1])
  x_max = np.max(dst[:, 0, 0])
  y_max = np.max(dst[:, 0, 1])

  # Get translation offset
  offset_x = int(-x_min)
  offset_y = int(-y_min)

  # Create new destination image with proper size
  h_new = int(y_max - y_min)
  w_new = int(x_max - x_min)
  dst_img = cv2.warpPerspective(img, M, (w_new + offset_x, h_new + offset_y))

  return dst_img, offset_x, offset_y

def blend_images(img1, img2, offset_x, offset_y):
  # Get image heights and widths
  h1, w1 = img1.shape[:2]
  h2, w2 = img2.shape[:2]

  # Create mask for blending (black for image1, white for image2)
  mask = np.zeros((h_new, w_new), np.float32)
  mask[offset_y:offset_y+h1, offset_x:offset_x+w1] = 1.0

  # Blend images using weighted average
  blended_img = (img1.astype(np.float32) * (1 - mask) + img2.astype(np.float32) * mask)

  return blended_img.astype(np.uint8)

def stitch_images(images):
  # Reference image for stitching
  reference_img = images[0]

  # Resulting panorama image
  panorama = None

  for i in range(1, len(images)):
    # Find ORB features and descriptors
    img1_kp, img1_des = find_orb_features(reference_img)
    img2_kp, img2_des = find_orb_features(images[i])

    # Match features
    matches = match_features(img1_des, img2_des)

